In [ ]:
!pip install -U transformers datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 81.5 MB/s eta 0:00:00


In [ ]:
# STEP 1: Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

# Adjust this path based on where you saved the copy
source_path = "/content/drive/My Drive/DataforDissertation/Contracts.txt"   # 👈 Your full folder
target_path = "/content/Contracts"

shutil.copytree(source_path, target_path)

print(f"✅ All files copied to: {target_path}")

✅ All files copied to: /content/Contracts


In [ ]:
import os
import pandas as pd

contracts_folder = '/content/Contracts'

category_keywords = {
    "IP": ["INTELLECTUAL PROPERTY AGREEMENT", "INTELLECTUAL PROPERTY RIGHTS AGREEMENT"],
    "COLLABORATION": ["COOPERATION AGREEMENT", "COLLABORATION AGREEMENT","Cooperation Agreement", "Collaborative Development and Commercialization Agreement", "Collaboration Agreement"],
    "AFFILIATE AGREEMENTS": ["AFFILIATE AGREEMENT", "Affiliate Program"],#"AFFILIATE"
    "AGENCY AGREEMENTS": ["AGENCY AGREEMENT", "Agency Agreement"],
    "CO-BRANDING": ["CO-BRANDING AGREEMENT","CO-BRANDING Agreement", "CO-BRANDING CONTENT", "CO-BRANDING"],
    "CONSULTING AGREEMENTS": ["CONSULTING AGREEMENT"],
    "DEVELOPMENT": ["DEVELOPMENT AGREEMENT", "Development Agreement", "DEVELOPMENT AND OPTION AGREEMENT"], #"DEVELOPMENT",
    "DISTRIBUTOR": ["DISTRIBUTOR AGREEMENT", "Distributor Agreement", "Distributorship agreement", "DISTRIBUTION AGREEMENT", "DISTRIBUTORSHIP AGREEMENT"],
    "ENDORSEMENT": ["ENDORSEMENT AGREEMENT", "Endorsement Agreement", "ENDORSEMENT"],
    "FRANCHISE": ["FRANCHISE AGREEMENT", "Franchisor", "Franchise Agreement"],
    "HOSTING": ["HOSTING AGREEMENT", "Hosting Agreement", "HOSTING", "ESCROW"],
    "JOINT VENTURE": ["JOINT VENTURE AGREEMENT", "JOINT FILING AGREEMENT", "JOINT FILING", "JOINT VENTURE"],
    "LICENSE AGREEMENTS": ["LICENSE AGREEMENT", "Content and License Agreement", "CONTENT LICENSE", "Content License Agreement", "LICENSING", "licensees"],
    "MAINTENANCE": ["Maintenance Agreement", "MAINTENANCE AGREEMENT"], #"Maintenance"]
    "MANUFACTURING": ["MANUFACTURING AGREEMENT", "MANUFACTURING", "Manufacturing Agreement"],
    "MARKETING": ["MARKETING AGREEMENT", "Marketing Agreement"],#"MARKETING",
    "NON-COMPETE NON-SOLICIT": ["NON-COMPETITION AND NON-SOLICITATION AGREEMENT", "NON-COMPETITION AGREEMENT", "NON COMPETITION AGREEMENT"],
    "OUTSOURCING": ["Outsourcing Agreement", "OUTSOURCING AGREEMENT", "Outsourcing Contract", "MANUFACTURING OUTSOURCING AGREEMENT"],
    "PROMOTION": ["PROMOTION AGREEMENT","Promotion Agreement"],
    "RESELLER": ["RESELLER AGREEMENT", "Reseller Agreement"],
    "SERVICE": ["SERVICES AGREEMENT", "SERVICING AGREEMENT", "SERVICE AGREEMENT", "Service Agreement", "Services Agreement"],
    "SPONSORSHIP": ["SPONSORSHIP AGREEMENT", "SPONSORSHIP", "Sponsorship Agreement"],
    "STRATEGIC ALLIANCE": ["STRATEGIC ALLIANCE AGREEMENT", "Strategic Alliance Agreement"],
    "SUPPLY": ["SUPPLY AGREEMENT", "Supply Agreement", "SUPPLY CONTRACT"],
    "TRANSPORTATION": ["Transportation Agreement", "TRANSPORTATION SERVICES AGREEMENT", "TRANSPORTATION", "TRANSPORTATION SERVICE AGREEMENT"]
}

# Prepare a list of (keyword, category) tuples, sorted by keyword length descending
keyword_cat_pairs = []
for category, keywords in category_keywords.items():
    for kw in keywords:
        keyword_cat_pairs.append((kw, category))
keyword_cat_pairs.sort(key=lambda x: len(x[0]), reverse=True)

def categorize_text_exact(text, keyword_cat_pairs, header_chars=2000):
    header = text[:header_chars].upper()  # Case-insensitive
    best_category = None
    best_idx = len(header) + 1
    for keyword, category in keyword_cat_pairs:
        idx = header.find(keyword.upper())
        if idx != -1 and idx < best_idx:
            best_idx = idx
            best_category = category
    return best_category if best_category else "Unknown"

data = []
for fname in os.listdir(contracts_folder):
    if fname.endswith('.txt'):
        file_path = os.path.join(contracts_folder, fname)
        with open(file_path, encoding='utf-8', errors='ignore') as f:
            text = f.read()
        cat = categorize_text_exact(text, keyword_cat_pairs)
        data.append({'file_name': fname, 'text': text, 'category': cat})

df = pd.DataFrame(data)
df = df[df['category'] != 'Unknown']

category2id = {cat: idx for idx, cat in enumerate(sorted(df['category'].unique()))}
df['label'] = df['category'].map(category2id)

df[['file_name','text', 'label']].to_csv('/content/contracts_xlnet_train.csv', index=False)

print("Label mapping:", category2id)
print(df.head())

Label mapping: {'AFFILIATE AGREEMENTS': 0, 'AGENCY AGREEMENTS': 1, 'CO-BRANDING': 2, 'COLLABORATION': 3, 'CONSULTING AGREEMENTS': 4, 'DEVELOPMENT': 5, 'DISTRIBUTOR': 6, 'ENDORSEMENT': 7, 'FRANCHISE': 8, 'HOSTING': 9, 'IP': 10, 'JOINT VENTURE': 11, 'LICENSE AGREEMENTS': 12, 'MAINTENANCE': 13, 'MANUFACTURING': 14, 'MARKETING': 15, 'NON-COMPETE NON-SOLICIT': 16, 'OUTSOURCING': 17, 'PROMOTION': 18, 'RESELLER': 19, 'SERVICE': 20, 'SPONSORSHIP': 21, 'STRATEGIC ALLIANCE': 22, 'SUPPLY': 23, 'TRANSPORTATION': 24}
                                           file_name  \
0  CardlyticsInc_20180112_S-1_EX-10.16_11002987_E...   
1  ULTRAGENYXPHARMACEUTICALINC_12_23_2013-EX-10.9...   
2  ON4COMMUNICATIONSINC_07_02_2009-EX-10.1-PROMOT...   
3  INTERNATIONALFASTFOODCORP_04_04_1997-EX-99-FRA...   
4  CORIOINC_07_20_2000-EX-10.5-LICENSE AND HOSTIN...   

                                                text     category  label  
0  SCHEDULE B\n\nCustomization Schedule\n\nThis C...  MAINTENANCE     13  
1  

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Use the df DataFrame created in the previous cell
df = pd.read_csv('/content/contracts_xlnet_train.csv')  # Change to the correct filename
print(df.head())

# Create the category2id mapping from the unique labels in the loaded df
id2category = {idx: cat for cat, idx in category2id.items()} # Reusing the id2category mapping from previous cell
df['category'] = df['label'].map(id2category)

category2id = {cat: idx for idx, cat in id2category.items()}


# Split: 80% train, 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Save for Hugging Face datasets
train_df[['file_name','text', 'label']].to_csv('/content/contracts_train.csv', index=False)
test_df[['file_name','text', 'label']].to_csv('/content/contracts_test.csv', index=False)

print("Label mapping (category to id):", category2id)
print("Label mapping (id to category):", id2category)

                                           file_name  \
0  CardlyticsInc_20180112_S-1_EX-10.16_11002987_E...   
1  ULTRAGENYXPHARMACEUTICALINC_12_23_2013-EX-10.9...   
2  ON4COMMUNICATIONSINC_07_02_2009-EX-10.1-PROMOT...   
3  INTERNATIONALFASTFOODCORP_04_04_1997-EX-99-FRA...   
4  CORIOINC_07_20_2000-EX-10.5-LICENSE AND HOSTIN...   

                                                text  label  
0  SCHEDULE B\n\nCustomization Schedule\n\nThis C...     13  
1  Exhibit 10.9\n\nSUPPLY AGREEMENT\n\nbetween\n\...     23  
2  925 West Georgia Street  Suite 1820  Vancouver...     18  
3  FRANCHISE AGREEMENT\n\n                       ...      8  
4  1                                             ...      9  
Label mapping (category to id): {'AFFILIATE AGREEMENTS': 0, 'AGENCY AGREEMENTS': 1, 'CO-BRANDING': 2, 'COLLABORATION': 3, 'CONSULTING AGREEMENTS': 4, 'DEVELOPMENT': 5, 'DISTRIBUTOR': 6, 'ENDORSEMENT': 7, 'FRANCHISE': 8, 'HOSTING': 9, 'IP': 10, 'JOINT VENTURE': 11, 'LICENSE AGREEMENTS': 12, 

In [ ]:
import datasets # Import datasets
from datasets import load_dataset, ClassLabel
import pandas as pd # Import pandas

# Load the datasets
dataset = load_dataset('csv', data_files={'train': '/content/contracts_train.csv', 'test': '/content/contracts_test.csv'})

# Convert the 'label' column to integer type in both splits
dataset['train'] = dataset['train'].cast_column('label', datasets.ClassLabel(names=sorted(df['category'].unique().tolist())))
dataset['test'] = dataset['test'].cast_column('label', datasets.ClassLabel(names=sorted(df['category'].unique().tolist())))

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Casting the dataset:   0%|          | 0/403 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
from transformers import XLNetTokenizer

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

Map:   0%|          | 0/403 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
from transformers import XLNetForSequenceClassification, Trainer, TrainingArguments
import datasets # Import the datasets library

num_labels = df['label'].nunique()  # number of unique categories

model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content/results_xlnet',
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    report_to="none"
)

# Ensure the label column is integer type
tokenized_datasets['train'] = tokenized_datasets['train'].cast_column('label', datasets.ClassLabel(num_classes=num_labels))
tokenized_datasets['test'] = tokenized_datasets['test'].cast_column('label', datasets.ClassLabel(num_classes=num_labels))


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Casting the dataset:   0%|          | 0/403 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/101 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,No log,3.081276
2,No log,2.806400
3,No log,2.290378
4,No log,1.969092
5,2.479700,1.851534


TrainOutput(global_step=505, training_loss=2.4694518127063714, metrics={'train_runtime': 242.8451, 'train_samples_per_second': 8.297, 'train_steps_per_second': 2.08, 'total_flos': 287071623621120.0, 'train_loss': 2.4694518127063714, 'epoch': 5.0})

In [ ]:
eval_result = trainer.evaluate()
print(eval_result)

{'eval_loss': 1.8515338897705078, 'eval_runtime': 2.123, 'eval_samples_per_second': 47.575, 'eval_steps_per_second': 12.247, 'epoch': 5.0}


In [ ]:
import torch

def predict_category(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}      # <-- Move inputs to GPU/CPU
    model.to(device)                                          # <-- Ensure model is on correct device
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = outputs.logits.argmax(dim=1).item()
    return prediction

In [ ]:
import torch
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): XLNetSequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    

In [ ]:
import pandas as pd

# Load your data (replace with your file if different)
df = pd.read_csv('/content/contracts_test.csv')  # or contracts_train.csv

# Ensure the label column is integer
df['label'] = df['label'].astype(int)

# Add a category string column from the label
df['Predicted Category'] = df['label'].map(id2category)

# Save to Excel
df.to_excel('/content/contracts_labels_with_category.xlsx', index=False)
print("Results saved to /content/contracts_labels_with_category.xlsx")

Results saved to /content/contracts_labels_with_category.xlsx


In [ ]:
df.columns

Index(['file_name', 'text', 'label', 'Predicted Category'], dtype='object')

In [ ]:
import torch
from tqdm.auto import tqdm
import pandas as pd

# Load your data
df = pd.read_csv('/content/contracts_test.csv')

# Make sure your XLNet model and tokenizer are loaded and on the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_category(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        pred = outputs.logits.argmax(dim=1).item()
    return pred

# Predict for each contract
predicted_labels = []
for text in tqdm(df['text']):
    pred_label = predict_category(text)
    predicted_labels.append(pred_label)

df['predicted_label'] = predicted_labels

  0%|          | 0/101 [00:00<?, ?it/s]

In [ ]:
df.to_excel('/content/contracts_with_xlnet_predictions.xlsx', index=False)
print("Results saved to /content/contracts_with_xlnet_predictions.xlsx")

Results saved to /content/contracts_with_xlnet_predictions.xlsx


In [ ]:
df.columns

Index(['file_name', 'text', 'label', 'predicted_label'], dtype='object')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd

# Load your results DataFrame (with 'label' and 'predicted_label')
df = pd.read_excel('/content/contracts_with_xlnet_predictions.xlsx')  # or .csv

# 1. Accuracy
acc = accuracy_score(df['label'], df['predicted_label'])
print("Accuracy: {:.2f}%".format(acc * 100))

# 2. Classification report (precision, recall, f1 for each category)
print("\nClassification Report:")
print(classification_report(df['label'], df['predicted_label']))

Accuracy: 49.50%

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.60      1.00      0.75         3
           2       0.43      0.75      0.55         4
           3       0.25      0.20      0.22         5
           4       0.67      1.00      0.80         2
           5       0.00      0.00      0.00         5
           6       0.35      1.00      0.52         6
           7       0.67      0.40      0.50         5
           8       0.00      0.00      0.00         3
           9       1.00      0.50      0.67         4
          10       0.00      0.00      0.00         3
          11       1.00      0.60      0.75         5
          12       0.45      0.71      0.56         7
          13       0.43      0.50      0.46         6
          14       0.50      0.75      0.60         4
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_true = df['label']
y_pred = df['predicted_label']

accuracy = accuracy_score(y_true, y_pred)*100
precision = precision_score(y_true, y_pred, average='weighted')*100
recall = recall_score(y_true, y_pred, average='weighted')*100
f1 = f1_score(y_true, y_pred, average='weighted')*100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1 Score: {f1:.2f}%")

Accuracy: 49.50%
Precision: 41.77%
Recall: 49.50%
F1 Score: 42.94%


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
